In [2]:
import pandas as pd 
df_master=pd.read_csv("../Processed Data/prd_master_orders_customers.csv")

In [3]:

df_master['order_purchase_timestamp'] = pd.to_datetime(df_master['order_purchase_timestamp'])



total_spend_per_cust = df_master.groupby('customer_unique_id')['total_payment_value'].sum().reset_index()
total_spend_per_cust.columns = ['customer_unique_id', 'total_lifetime_spend']


df_regression = df_master[df_master['is_first_order'] == 1].copy()


df_regression = df_regression.merge(total_spend_per_cust, on='customer_unique_id', how='left')


df_regression['label_next_order_value'] = (
    df_regression['total_lifetime_spend'] - df_regression['total_payment_value']
)



df_regression['label_next_order_value'] = df_regression['label_next_order_value'].fillna(0).clip(lower=0)

In [4]:
# Create the "Repeaters Only" dataset for regression
df_repeaters_only = df_regression[df_regression['label_next_order_value'] > 0].copy()

print(f"Total Rows: {len(df_regression)}")
print(f"Repeaters Rows: {len(df_repeaters_only)}")

Total Rows: 94990
Repeaters Rows: 2888


In [5]:
df_repeaters_only.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2888 entries, 106 to 94944
Data columns (total 54 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   customer_id                 2888 non-null   object        
 1   order_id                    2888 non-null   object        
 2   order_purchase_timestamp    2888 non-null   datetime64[ns]
 3   snapshot_date               2888 non-null   object        
 4   is_repeat_customer          2888 non-null   int64         
 5   past_order_count            2888 non-null   int64         
 6   days_since_last_order       2888 non-null   int64         
 7   is_first_order              2888 non-null   int64         
 8   customer_unique_id          2888 non-null   object        
 9   customer_zip_code_prefix    2888 non-null   int64         
 10  customer_city               2888 non-null   object        
 11  customer_state              2888 non-null   object        

In [6]:
df_repeaters_only.to_csv("../Processed Data/prd_loyal_customers_ordervalue.csv", index=False)